## Understanding Query Segmentation in RAG Applications
### What is Query Segmentation?
- Query segmentation (also called query decomposition or multi-query generation) is the process of breaking down complex, multi-faceted user queries into smaller, focused sub-queries. Each sub-query targets a specific aspect of the original question, allowing for more precise retrieval and comprehensive coverage of the user's information needs.
### When is Query Segmentation Done?
Query segmentation occurs during the query processing phase of RAG:

- Query Analysis → Detect if query is complex/multi-faceted
- 🔄 Query Segmentation → Break down into focused sub-queries
- Parallel Retrieval → Search for each sub-query independently
- Result Aggregation → Combine and deduplicate results
- Optional Reranking → Rank combined results
- Generation → LLM uses comprehensive context

### Why Query Segmentation Matters
Complex queries often contain multiple concepts that require different retrieval strategies:

- Multi-topic queries - spanning different domains
- Comparative questions - requiring information about multiple entities
- Step-by-step processes - needing sequential information
- Conditional queries - with multiple scenarios or constraints

## Key Differences in Output Quality
### Without Query Segmentation:

- Coverage Gaps: May miss important aspects of complex questions
- Bias Issues: Results skewed toward dominant terms in the query
- Lower Precision: General documents rank higher than specific ones
- Incomplete Answers: LLM receives context that doesn't cover all user needs

### With Query Segmentation:

- Comprehensive Coverage: Each aspect of the question gets dedicated retrieval
- Balanced Results: Equal attention to all parts of the complex query
- Higher Precision: Specific documents for each sub-topic
- Better Context: LLM receives more complete and relevant information

## Technical Implementation Details

In [ ]:
#Approach 1. LLM-Based Segmentation

pythondef segment_query_with_llm(query):
    prompt = f"""
    Break down this complex query into focused sub-queries:
    "{query}"
    
    Generate 3-5 specific sub-queries that cover different aspects.
    Each should be a complete, searchable question.
    """
    return llm.generate(prompt)

In [ ]:
#Approach 2 : Template Based Patterns

comparison_patterns = [
    r"compare (.*) and (.*)",
    r"(.*) vs (.*)",
    r"difference between (.*) and (.*)"
]

instruction_patterns = [
    r"how to (.*) and (.*)",
    r"steps to (.*) and (.*)",
    r"explain (.*) and provide (.*)"
]

In [ ]:
# Approach 3: Result Aggregation Strategies 

def aggregate_segmented_results(segment_results):
    # Combine results from all segments
    all_docs = {}
    for segment_idx, results in enumerate(segment_results):
        for doc in results:
            doc_id = doc['id']
            if doc_id not in all_docs:
                all_docs[doc_id] = doc
                all_docs[doc_id]['segment_scores'] = []
            all_docs[doc_id]['segment_scores'].append({
                'segment': segment_idx,
                'score': doc['score']
            })
    
    # Calculate final scores
    for doc in all_docs.values():
        doc['final_score'] = max(s['score'] for s in doc['segment_scores'])
    
    return sorted(all_docs.values(), key=lambda x: x['final_score'], reverse=True)

## When to Use Query Segmentation?
### Good Candidates:

- Multi-topic questions (compare A and B)
- Step-by-step requests (how to X and Y)
- Questions with multiple constraints
- Queries with conjunctions ("and", "or", "but")

### Skip Segmentation For:

- Simple, focused questions
- Single-concept queries
- When computational cost is a concern
- Very short queries



In [ ]:
import React, { useState } from 'react';
import { Search, Scissors, ArrowRight, CheckCircle, AlertTriangle, Lightbulb } from 'lucide-react';

const QuerySegmentationDemo = () => {
  const [activeTab, setActiveTab] = useState('without');
  const [selectedQuery, setSelectedQuery] = useState(0);
  
  // Demo queries with varying complexity
  const complexQueries = [
    {
      original: "Compare TensorFlow and PyTorch for computer vision tasks, explain their pros and cons, and provide installation instructions for both",
      segments: [
        "TensorFlow computer vision capabilities and features",
        "PyTorch computer vision capabilities and features", 
        "TensorFlow vs PyTorch comparison advantages disadvantages",
        "TensorFlow installation guide setup instructions",
        "PyTorch installation guide setup instructions"
      ],
      complexity: "High",
      topics: ["Framework Comparison", "Computer Vision", "Installation"]
    },
    {
      original: "What are the best practices for training transformer models and how do I optimize memory usage during training?",
      segments: [
        "transformer model training best practices techniques",
        "memory optimization strategies transformer training",
        "GPU memory management deep learning",
        "transformer training hyperparameters tuning"
      ],
      complexity: "Medium",
      topics: ["Training", "Optimization", "Memory Management"]
    },
    {
      original: "Explain gradient descent, its variants, and when to use each one",
      segments: [
        "gradient descent algorithm explanation",
        "stochastic gradient descent SGD",
        "mini-batch gradient descent",
        "Adam optimizer characteristics",
        "gradient descent variants comparison use cases"
      ],
      complexity: "Medium",
      topics: ["Algorithms", "Optimization", "Comparison"]
    }
  ];

  // Demo knowledge base documents
  const documents = [
    {
      id: 1,
      title: "TensorFlow for Computer Vision: Complete Guide",
      content: "TensorFlow provides excellent computer vision capabilities through tf.keras.applications with pre-trained models like ResNet, VGG, and InceptionV3. TensorFlow Lite enables mobile deployment.",
      relevance: {
        original: { q0: 0.65, q1: 0.25, q2: 0.30 },
        segments: { 
          q0: [0.95, 0.15, 0.70, 0.88, 0.10], 
          q1: [0.20, 0.15, 0.10, 0.25],
          q2: [0.85, 0.15, 0.20, 0.15, 0.10]
        }
      },
      category: "Framework Guide"
    },
    {
      id: 2,
      title: "PyTorch Computer Vision with torchvision",
      content: "PyTorch offers dynamic computation graphs ideal for computer vision research. Torchvision provides datasets, transforms, and pre-trained models. Easy debugging and experimentation.",
      relevance: {
        original: { q0: 0.62, q1: 0.22, q2: 0.28 },
        segments: { 
          q0: [0.18, 0.92, 0.75, 0.12, 0.85], 
          q1: [0.18, 0.12, 0.08, 0.20],
          q2: [0.15, 0.88, 0.18, 0.12, 0.08]
        }
      },
      category: "Framework Guide"
    },
    {
      id: 3,
      title: "Framework Comparison: TensorFlow vs PyTorch",
      content: "TensorFlow excels in production deployment with TensorFlow Serving and TensorFlow Lite. PyTorch offers superior research flexibility and easier debugging. Both support distributed training.",
      relevance: {
        original: { q0: 0.58, q1: 0.15, q2: 0.25 },
        segments: { 
          q0: [0.25, 0.28, 0.95, 0.20, 0.18], 
          q1: [0.12, 0.10, 0.08, 0.15],
          q2: [0.20, 0.22, 0.92, 0.10, 0.08]
        }
      },
      category: "Comparison"
    },
    {
      id: 4,
      title: "Installing TensorFlow: Step-by-Step Guide",
      content: "Install TensorFlow using pip install tensorflow. For GPU support, install tensorflow-gpu and ensure CUDA drivers are configured. Virtual environments recommended.",
      relevance: {
        original: { q0: 0.35, q1: 0.18, q2: 0.15 },
        segments: { 
          q0: [0.15, 0.12, 0.25, 0.95, 0.20], 
          q1: [0.15, 0.12, 0.10, 0.18],
          q2: [0.10, 0.08, 0.15, 0.12, 0.05]
        }
      },
      category: "Installation"
    },
    {
      id: 5,
      title: "PyTorch Installation and Setup",
      content: "Install PyTorch from pytorch.org with CUDA support selection. Use conda install pytorch torchvision torchaudio for complete setup. Verify installation with torch.cuda.is_available().",
      relevance: {
        original: { q0: 0.33, q1: 0.20, q2: 0.12 },
        segments: { 
          q0: [0.12, 0.18, 0.22, 0.20, 0.92], 
          q1: [0.18, 0.15, 0.12, 0.20],
          q2: [0.08, 0.10, 0.12, 0.15, 0.05]
        }
      },
      category: "Installation"
    },
    {
      id: 6,
      title: "Transformer Training Best Practices",
      content: "Use learning rate warmup, gradient clipping, and appropriate batch sizes for transformer training. Layer normalization and residual connections are crucial for stability.",
      relevance: {
        original: { q0: 0.25, q1: 0.88, q2: 0.45 },
        segments: { 
          q0: [0.20, 0.15, 0.18, 0.12, 0.10], 
          q1: [0.95, 0.25, 0.18, 0.88],
          q2: [0.15, 0.20, 0.25, 0.12, 0.18]
        }
      },
      category: "Training"
    },
    {
      id: 7,
      title: "Memory Optimization for Large Models",
      content: "Use gradient checkpointing, mixed precision training, and model parallelism to reduce memory usage. DeepSpeed and FairScale provide memory optimization techniques.",
      relevance: {
        original: { q0: 0.22, q1: 0.85, q2: 0.35 },
        segments: { 
          q0: [0.15, 0.12, 0.18, 0.10, 0.08], 
          q1: [0.30, 0.92, 0.88, 0.25],
          q2: [0.12, 0.18, 0.22, 0.15, 0.10]
        }
      },
      category: "Optimization"
    },
    {
      id: 8,
      title: "Gradient Descent Algorithm Fundamentals",
      content: "Gradient descent minimizes loss functions by iteratively moving in the direction of steepest descent. The learning rate controls step size and convergence behavior.",
      relevance: {
        original: { q0: 0.30, q1: 0.42, q2: 0.85 },
        segments: { 
          q0: [0.18, 0.15, 0.20, 0.12, 0.10], 
          q1: [0.35, 0.25, 0.20, 0.30],
          q2: [0.95, 0.25, 0.20, 0.18, 0.30]
        }
      },
      category: "Algorithm"
    },
    {
      id: 9,
      title: "SGD vs Adam vs RMSprop: Optimizer Comparison",
      content: "SGD with momentum works well for simple problems. Adam adapts learning rates per parameter. RMSprop handles sparse gradients effectively. Choose based on problem characteristics.",
      relevance: {
        original: { q0: 0.28, q1: 0.45, q2: 0.82 },
        segments: { 
          q0: [0.15, 0.12, 0.18, 0.10, 0.08], 
          q1: [0.32, 0.28, 0.25, 0.35],
          q2: [0.30, 0.85, 0.28, 0.92, 0.88]
        }
      },
      category: "Comparison"
    }
  ];

  const currentQuery = complexQueries[selectedQuery];
  const queryKey = `q${selectedQuery}`;

  // Get results without segmentation (single query)
  const withoutSegmentation = [...documents]
    .sort((a, b) => b.relevance.original[queryKey] - a.relevance.original[queryKey])
    .slice(0, 5);

  // Get results with segmentation (multiple sub-queries)
  const withSegmentation = () => {
    const allResults = new Map();
    
    currentQuery.segments.forEach((segment, segIdx) => {
      documents.forEach(doc => {
        const score = doc.relevance.segments[queryKey][segIdx];
        const existing = allResults.get(doc.id);
        if (!existing || score > existing.score) {
          allResults.set(doc.id, {
            ...doc,
            score: score,
            matchedSegment: segment,
            segmentIndex: segIdx
          });
        }
      });
    });

    return Array.from(allResults.values())
      .sort((a, b) => b.score - a.score)
      .slice(0, 6);
  };

  const segmentedResults = withSegmentation();

  const ScoreBar = ({ score, color = "blue", label }) => (
    <div className="mb-1">
      <div className="flex justify-between text-xs mb-1">
        <span>{label}</span>
        <span>{score.toFixed(3)}</span>
      </div>
      <div className="w-full bg-gray-200 rounded-full h-2">
        <div 
          className={`bg-${color}-500 h-2 rounded-full transition-all duration-300`} 
          style={{ width: `${score * 100}%` }}
        />
      </div>
    </div>
  );

  const DocumentCard = ({ doc, rank, searchType, showSegment = false }) => (
    <div className="border rounded-lg p-4 mb-3 bg-white shadow-sm hover:shadow-md transition-shadow">
      <div className="flex justify-between items-start mb-2">
        <div className="flex items-center gap-2">
          <span className="text-lg font-bold text-gray-600">#{rank}</span>
          <div className="flex-1">
            <h3 className="font-semibold text-gray-800">{doc.title}</h3>
            <span className="text-xs bg-gray-100 px-2 py-1 rounded text-gray-600">
              {doc.category}
            </span>
            {showSegment && doc.matchedSegment && (
              <div className="mt-1">
                <span className="text-xs bg-green-100 text-green-700 px-2 py-1 rounded">
                  Matched: "{doc.matchedSegment.substring(0, 50)}..."
                </span>
              </div>
            )}
          </div>
        </div>
        <div className="text-right w-32">
          {searchType === 'original' && (
            <ScoreBar 
              score={doc.relevance.original[queryKey]} 
              color="blue" 
              label="Relevance"
            />
          )}
          {searchType === 'segmented' && (
            <ScoreBar 
              score={doc.score} 
              color="green" 
              label="Best Match"
            />
          )}
        </div>
      </div>
      <p className="text-gray-600 text-sm leading-relaxed">{doc.content}</p>
    </div>
  );

  const SegmentCard = ({ segment, index }) => (
    <div className="bg-blue-50 border border-blue-200 rounded-lg p-3 mb-2">
      <div className="flex items-start gap-2">
        <span className="bg-blue-500 text-white rounded-full w-6 h-6 flex items-center justify-center text-xs font-bold">
          {index + 1}
        </span>
        <div className="flex-1">
          <p className="text-sm text-blue-800 font-medium">"{segment}"</p>
        </div>
      </div>
    </div>
  );

  return (
    <div className="max-w-7xl mx-auto p-6 bg-gray-50 min-h-screen">
      <h1 className="text-3xl font-bold text-center mb-2">RAG Query Segmentation Demo</h1>
      
      {/* Query Selection */}
      <div className="mb-6">
        <h2 className="text-lg font-semibold mb-3 text-center">Choose a Complex Query:</h2>
        <div className="grid grid-cols-1 gap-4">
          {complexQueries.map((query, idx) => (
            <button
              key={idx}
              onClick={() => setSelectedQuery(idx)}
              className={`p-4 rounded-lg border-2 text-left transition-all ${
                selectedQuery === idx
                  ? 'border-blue-500 bg-blue-50'
                  : 'border-gray-200 bg-white hover:border-gray-300'
              }`}
            >
              <div className="flex justify-between items-start mb-2">
                <div className="font-medium text-gray-800">
                  "{query.original}"
                </div>
                <div className="flex gap-1 ml-4">
                  <span className={`text-xs px-2 py-1 rounded ${
                    query.complexity === 'High' ? 'bg-red-100 text-red-700' : 'bg-yellow-100 text-yellow-700'
                  }`}>
                    {query.complexity}
                  </span>
                </div>
              </div>
              <div className="flex flex-wrap gap-1">
                {query.topics.map((topic, tidx) => (
                  <span key={tidx} className="text-xs bg-blue-100 text-blue-700 px-2 py-1 rounded">
                    {topic}
                  </span>
                ))}
              </div>
            </button>
          ))}
        </div>
      </div>

      {/* Current Query Display */}
      <div className="text-center mb-6">
        <div className="bg-blue-100 p-3 rounded-lg inline-block max-w-4xl">
          <Search className="inline mr-2" size={18} />
          <span className="font-medium">Active Query: "{currentQuery.original}"</span>
        </div>
      </div>

      {/* Segmentation Preview */}
      <div className="mb-6 bg-white rounded-lg shadow-sm p-6">
        <div className="flex items-center gap-2 mb-4">
          <Scissors className="text-blue-500" size={20} />
          <h3 className="text-lg font-semibold">Query Segmentation Analysis</h3>
        </div>
        <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
          <div>
            <h4 className="font-medium mb-3 text-gray-700">Original Complex Query:</h4>
            <div className="bg-gray-100 p-3 rounded border-l-4 border-gray-400">
              <p className="text-sm italic">"{currentQuery.original}"</p>
            </div>
          </div>
          <div>
            <h4 className="font-medium mb-3 text-blue-700">Generated Sub-queries:</h4>
            <div className="space-y-2">
              {currentQuery.segments.map((segment, idx) => (
                <SegmentCard key={idx} segment={segment} index={idx} />
              ))}
            </div>
          </div>
        </div>
      </div>

      {/* Method Tabs */}
      <div className="flex justify-center mb-6">
        <div className="bg-white rounded-lg p-1 shadow-sm">
          <button
            onClick={() => setActiveTab('without')}
            className={`px-4 py-2 rounded-md mr-2 transition-colors flex items-center gap-2 ${
              activeTab === 'without' 
                ? 'bg-blue-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <AlertTriangle size={16} />
            Without Segmentation
          </button>
          <button
            onClick={() => setActiveTab('with')}
            className={`px-4 py-2 rounded-md mr-2 transition-colors flex items-center gap-2 ${
              activeTab === 'with' 
                ? 'bg-green-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <CheckCircle size={16} />
            With Segmentation
          </button>
          <button
            onClick={() => setActiveTab('comparison')}
            className={`px-4 py-2 rounded-md transition-colors flex items-center gap-2 ${
              activeTab === 'comparison' 
                ? 'bg-purple-500 text-white' 
                : 'text-gray-600 hover:bg-gray-100'
            }`}
          >
            <ArrowRight size={16} />
            Side-by-Side
          </button>
        </div>
      </div>

      {/* Results Display */}
      {activeTab === 'without' && (
        <div className="bg-white rounded-lg shadow-sm p-6">
          <div className="flex items-center gap-2 mb-4">
            <AlertTriangle className="text-blue-500" />
            <h2 className="text-xl font-semibold text-blue-600">
              Results: Single Query Search
            </h2>
          </div>
          <div className="bg-yellow-50 border border-yellow-200 rounded p-3 mb-4">
            <p className="text-yellow-800 text-sm">
              <strong>Limitation:</strong> Single query retrieval may miss important aspects of complex questions, 
              leading to incomplete or biased results that don't cover all user information needs.
            </p>
          </div>
          {withoutSegmentation.map((doc, index) => (
            <DocumentCard 
              key={doc.id} 
              doc={doc} 
              rank={index + 1} 
              searchType="original"
            />
          ))}
        </div>
      )}

      {activeTab === 'with' && (
        <div className="bg-white rounded-lg shadow-sm p-6">
          <div className="flex items-center gap-2 mb-4">
            <CheckCircle className="text-green-500" />
            <h2 className="text-xl font-semibold text-green-600">
              Results: Query Segmentation
            </h2>
          </div>
          <div className="bg-green-50 border border-green-200 rounded p-3 mb-4">
            <p className="text-green-800 text-sm">
              <strong>Advantage:</strong> Each sub-query retrieves documents optimized for specific aspects, 
              providing comprehensive coverage and better relevance across all parts of the complex question.
            </p>
          </div>
          {segmentedResults.map((doc, index) => (
            <DocumentCard 
              key={doc.id} 
              doc={doc} 
              rank={index + 1} 
              searchType="segmented"
              showSegment={true}
            />
          ))}
        </div>
      )}

      {activeTab === 'comparison' && (
        <div className="grid grid-cols-1 lg:grid-cols-2 gap-6">
          <div className="bg-white rounded-lg shadow-sm p-6">
            <div className="flex items-center gap-2 mb-4">
              <AlertTriangle className="text-blue-500" />
              <h3 className="text-lg font-semibold text-blue-600">Without Segmentation</h3>
            </div>
            <div className="mb-4 text-sm text-gray-600">
              Single query: "{currentQuery.original.substring(0, 60)}..."
            </div>
            {withoutSegmentation.map((doc, index) => (
              <div key={doc.id} className="border rounded p-3 mb-2">
                <div className="font-medium text-sm">#{index + 1} {doc.title}</div>
                <div className="text-blue-600 text-xs">
                  Score: {doc.relevance.original[queryKey].toFixed(3)}
                </div>
                <div className="text-xs text-gray-500">{doc.category}</div>
              </div>
            ))}
          </div>
          
          <div className="bg-white rounded-lg shadow-sm p-6">
            <div className="flex items-center gap-2 mb-4">
              <CheckCircle className="text-green-500" />
              <h3 className="text-lg font-semibold text-green-600">With Segmentation</h3>
            </div>
            <div className="mb-4 text-sm text-gray-600">
              {currentQuery.segments.length} focused sub-queries
            </div>
            {segmentedResults.map((doc, index) => (
              <div key={doc.id} className="border rounded p-3 mb-2">
                <div className="font-medium text-sm">#{index + 1} {doc.title}</div>
                <div className="text-green-600 text-xs">
                  Score: {doc.score.toFixed(3)}
                </div>
                <div className="text-xs text-gray-500">{doc.category}</div>
                {doc.matchedSegment && (
                  <div className="text-xs text-green-700 mt-1">
                    Best match: "{doc.matchedSegment.substring(0, 40)}..."
                  </div>
                )}
              </div>
            ))}
          </div>
        </div>
      )}

      {/* Analysis Section */}
      <div className="mt-8 bg-yellow-50 border border-yellow-200 rounded-lg p-6">
        <div className="flex items-center gap-2 mb-3">
          <Lightbulb className="text-yellow-600" />
          <h3 className="text-lg font-semibold text-yellow-800">
            Coverage Analysis: "{currentQuery.original.substring(0, 80)}..."
          </h3>
        </div>
        <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
          <div>
            <h4 className="font-medium text-red-700 mb-2">Without Segmentation Issues:</h4>
            <ul className="space-y-1 text-sm text-red-600">
              <li>• May miss specific technical details</li>
              <li>• Biased toward dominant query terms</li>
              <li>• Incomplete coverage of multi-faceted questions</li>
              <li>• Lower precision for specific sub-topics</li>
              <li>• May prioritize general over specific content</li>
            </ul>
          </div>
          <div>
            <h4 className="font-medium text-green-700 mb-2">With Segmentation Benefits:</h4>
            <ul className="space-y-1 text-sm text-green-600">
              <li>• Comprehensive coverage of all aspects</li>
              <li>• Higher precision for each sub-topic</li>
              <li>• Better balance between different concepts</li>
              <li>• Finds specialized content for each aspect</li>
              <li>• Reduces query ambiguity effects</li>
            </ul>
          </div>
        </div>
      </div>

      {/* Implementation Details */}
      <div className="mt-6 bg-gray-100 rounded-lg p-6">
        <h3 className="text-lg font-semibold mb-3">Query Segmentation Implementation:</h3>
        <div className="grid grid-cols-1 md:grid-cols-2 gap-6">
          <div>
            <h4 className="font-medium mb-2">Segmentation Strategies:</h4>
            <ul className="space-y-2 text-sm">
              <li><strong>LLM-based:</strong> Use GPT/Claude to break down queries</li>
              <li><strong>Template-based:</strong> Pattern matching for common structures</li>
              <li><strong>Keyword extraction:</strong> Identify distinct concepts/entities</li>
              <li><strong>Dependency parsing:</strong> Analyze grammatical structure</li>
            </ul>
          </div>
          <div>
            <h4 className="font-medium mb-2">Result Aggregation:</h4>
            <ul className="space-y-2 text-sm">
              <li><strong>Score combination:</strong> Max, sum, or weighted average</li>
              <li><strong>Deduplication:</strong> Remove duplicate documents</li>
              <li><strong>Diversity:</strong> Ensure coverage of all segments</li>
              <li><strong>Reranking:</strong> Final relevance scoring</li>
            </ul>
          </div>
        </div>
      </div>
    </div>
  );
};

export default QuerySegmentationDemo;